# <img src="PML_Logo1.jpg">

# Pre-Interview Visualisation Task - Benjamin O'Driscoll

#### Reference Site: Western Channel Observatory <br> Coastal Station: L4 <br> Instrument: CTD 

The aim of this notebook is to visualise data from the WCO, L4, CTD in-situ time series data file.
<br>
A CTD instrument is used to take measurements of several variables over depth for the study of seasonal and
annual trends. <br>
A link to the data used in this notebook is provided [HERE](https://www.dropbox.com/s/nz2xotqglirl0wy/L4_CTDf_ODV_format.txt?dl=0)

#### Use of the Bokeh Library

The visualisation package used in this notebook is the Bokeh library. It offers the following advantages for the benefit of the end user 

- Interactive Plots 
- Fully Cusotmisable Appearances
- Plots which are Linked Together
- Hover tools which offer Additional Information relating to Data
- Automatic HTML Generation for Easier Webpage Integration 


#### Note to the User

Although this notebook has been designed to be scalable there are some nasty hardcoded parts which should be removed for future evolutions. These parts are highlighted at the end of the notebook.



# 1) Notebook Setup

### Stage 1 - Module Import

In [1]:
# Import modules required for Jupyter notebook

import numpy as np
import pandas as pd
import datetime

from datetime import datetime
from scipy import stats

from bokeh.plotting import figure,show, output_file, ColumnDataSource
from bokeh.models import Range1d, LinearColorMapper, DateRangeSlider, RangeSlider, CDSView, CustomJSFilter, CustomJS, BoxAnnotation, ColorBar
from bokeh.io import output_notebook
from bokeh.layouts import row, gridplot, column, layout

from bokeh.core.validation import silence
from bokeh.core.validation.warnings import FIXED_SIZING_MODE


### Stage 2 - Define Functions

These are functions that are called throughout this notebook are defined here so that the flow of the notebook is not interupted

In [2]:
def columnNames(name):
    
    """ Format of the raw text file has the name and unit of the header on two rows.
    During import using pandas, dataFrame forms these into a multiple index.
    When these are flattened into 1 header name there is no parenthesis around the units.
    
    This function changes the name of the column headers. 
    FROM: "NAME UNIT"
    TO: "NAME (UNIT)"
    """
    # Select the NAME
    firstPart=name[0:name.find(' ')+1:]
    # Select the UNIT
    secondPart=name[name.find(' ')+1:]
    # Add parentheses around the UNIT 
    newName=firstPart+'('+secondPart+')'
    
    return newName

In [3]:
def dateTimeCorrection(dateTimeEntry):
    """Format of most date/time stamps are 23 characters long.
    There are a few date/time stamps that are 24 characters long as they have an additional
    character in the 's' part of the timestamp.
    
    This function truncates the length of the timestamp to 23 characters long.
    This prevents errors handling these values downstream
    
    FROM: "2002-01-07T12:00:000.000"
    TO: "2002-01-07T12:00:00.000"    
    """
    
    if len(dateTimeEntry) > 23:
        # If is is larger than truncate the length of the date/time stamp
        dateTimeEntry = dateTimeEntry[0:19] + ".000"
        return dateTimeEntry
    else:
        # Do nothing
        return (dateTimeEntry)
        

# 2) Data Wrangling

### Stage 1 - Data Import

Bring the data from its raw form into this notebook in an efficient manner

In [4]:
# Need to tell pandas what the datatype of each column is. If we didn't then the pandas
# would have to make a guess for each column which is inefficient on memory. 
# datetypeDict is a dictionary which warns pandas what to expect when importing
datatypeDict={'Lat':float, 'Lon':float, 'Depth':float,
              'Temperature':float, 'Transmission':float,
              'PAR':float, 'Fluorescence':float, 'Density':float,
              'Oxygen':float, 'Salinity':float, 'Turbidity':float} 

# Need to explicitly tell pandas the delimiter is whitespace and headers are on two rows.
# Dictionary defined above is used to improve memory efficiency.  
# Commented out line below was used for development purposes. It uses a file with only ~2500 rows 
# dataFrame = pd.read_csv('L4_CTDf_ODV_formatSmall.txt', delim_whitespace=True,
#                         dtype=datatypeDict, header=[0,1])
dataFrame = pd.read_csv('L4_CTDf_ODV_format.txt', delim_whitespace=True,
                        dtype=datatypeDict, header=[0,1])

In [5]:
# Take a sneek peak at the data
display(dataFrame.head())

,Cruise,Station,Type,DateTime,Lat,Lon,Depth,Temperature,Transmission,PAR,Fluorescence,Density,Oxygen,Salinity,Turbidity
,CCC,SS,T,yyyy-mm-ddThh:mm:ss.sss,degN,degE,m,degC,%,uE/m2/s,volts,kg/m3,uM,PSU,NTU
0,WCO,L4,c,2002-01-07T12:00:00.000,50.25174,-4.22056,48.0,10.6825,NaN,NaN,2.0286,1027.2241,NaN,35.2221,NaN
1,WCO,L4,c,2002-01-07T12:00:00.000,50.25174,-4.22056,47.5,10.6827,NaN,NaN,2.0396,1027.2219,NaN,35.2222,NaN
2,WCO,L4,c,2002-01-07T12:00:00.000,50.25174,-4.22056,47.0,10.6826,NaN,NaN,1.9936,1027.2198,NaN,35.2224,NaN
3,WCO,L4,c,2002-01-07T12:00:00.000,50.25174,-4.22056,46.5,10.6828,NaN,NaN,2.0387,1027.2175,NaN,35.2224,NaN
4,WCO,L4,c,2002-01-07T12:00:00.000,50.25174,-4.22056,46.0,10.6826,NaN,NaN,2.0234,1027.2152,NaN,35.2224,NaN


### Stage 2 - Data Transformation
Transform the data so that it is in a format that can be handled easily downstream

In [6]:
# This line joins the NAME header with the UNIT header which 
# simplifies the dataFrame for manipulation as only calling one header is easier 
dataFrame.columns=dataFrame.columns.map(' '.join) 

# This list comprehension builds up a new list of columns names by adding the 
# UNIT values into parantheses 
newColumnNames=[columnNames(i) for i in dataFrame.columns] 

# This new list is then set as the column names
dataFrame.columns=newColumnNames 

In [7]:
# Find the times where the length of the timestamp is erroneously not 23.
# These will form part of the output to feedback to device owner. 
longTimes=dataFrame[dataFrame['DateTime (yyyy-mm-ddThh:mm:ss.sss)'].apply(lambda x: len(x)>23)].iloc[:,3]
errorFrameTimes=longTimes.to_frame()

In [8]:
# This cell loops over the column values and truncates them to the correct size
dataFrame['DateTime (yyyy-mm-ddThh:mm:ss.sss)'] = [dateTimeCorrection(entry) for entry in dataFrame['DateTime (yyyy-mm-ddThh:mm:ss.sss)']]

# Line below adds another column to the dataFrame which copies all of the string values.
# This makes accessing date/time stamp values easier when hovering in later plots. 
dataFrame['DateTimeLabels']=dataFrame['DateTime (yyyy-mm-ddThh:mm:ss.sss)'] 

In [9]:
# DateTime column is imported as string which will cause problems downstream. 
# Convert this column to DateTime datatype here:
dataFrame.iloc[:,3] =  pd.to_datetime(dataFrame.iloc[:,3], format='%Y-%m-%dT%H:%M:%S.%f')


In [10]:
# This cell converts the DateTime column into a column of Timestamps. 
# These are easier to handle downstream for visualisations

timeStampList=[datetime.timestamp(entry)*1000 for entry in dataFrame['DateTime (yyyy-mm-ddThh:mm:ss.sss)']]
dataFrame['Timestamp'] = pd.DataFrame(timeStampList)

display(dataFrame.head())

,Cruise (CCC),Station (SS),Type (T),DateTime (yyyy-mm-ddThh:mm:ss.sss),Lat (degN),Lon (degE),Depth (m),Temperature (degC),Transmission (%),PAR (uE/m2/s),Fluorescence (volts),Density (kg/m3),Oxygen (uM),Salinity (PSU),Turbidity (NTU),DateTimeLabels,Timestamp
0,WCO,L4,c,2002-01-07 12:00:00,50.25174,-4.22056,48.0,10.6825,NaN,NaN,2.0286,1027.2241,NaN,35.2221,NaN,2002-01-07T12:00:00.000,1.010405e+12
1,WCO,L4,c,2002-01-07 12:00:00,50.25174,-4.22056,47.5,10.6827,NaN,NaN,2.0396,1027.2219,NaN,35.2222,NaN,2002-01-07T12:00:00.000,1.010405e+12
2,WCO,L4,c,2002-01-07 12:00:00,50.25174,-4.22056,47.0,10.6826,NaN,NaN,1.9936,1027.2198,NaN,35.2224,NaN,2002-01-07T12:00:00.000,1.010405e+12
3,WCO,L4,c,2002-01-07 12:00:00,50.25174,-4.22056,46.5,10.6828,NaN,NaN,2.0387,1027.2175,NaN,35.2224,NaN,2002-01-07T12:00:00.000,1.010405e+12
4,WCO,L4,c,2002-01-07 12:00:00,50.25174,-4.22056,46.0,10.6826,NaN,NaN,2.0234,1027.2152,NaN,35.2224,NaN,2002-01-07T12:00:00.000,1.010405e+12


# 3) Data Analysis

## Exploratory Data Analysis
### Selecting Variables of Interest

Determine a relationship between variables that could be interesting to plot.
<br>Use the `corr()` method to automatically determine correlations between two variables

In [11]:
# The corr() method determines the (Pearson) correlation between all columns of a dataFrame 
correlatedTable=dataFrame.corr()

# Unstack from dataFrame to Series
correlatedPairs=correlatedTable.unstack()

# Sort them in order and then drop all NaN values as we are not interested in these
sortedPairs = correlatedPairs.sort_values(kind="quicksort") 
sortedPairsNoNan=sortedPairs.dropna()

# Using the .corr() method will give the value of 1 along the diagonals of the 
# where each column is correlated 'perfectly' with itself. We want to ignore these.
# To do this, we index the values between 0.5 and 1.

strongPositiveCorrelationsRaw=sortedPairsNoNan[sortedPairsNoNan.between(0.5,1, inclusive=False)]

# Since every row is correlated with every column and every column is correlated with 
# every row, we get repeated values. Line below selects remove duplicate values.
strongPositiveCorrelations=strongPositiveCorrelationsRaw[0:-1:2]

# Display all variables with Pearson correlation between 0.5 and 1
display(strongPositiveCorrelations)

Turbidity (NTU)  Fluorescence (volts)    0.503237
Salinity (PSU)   Density (kg/m3)         0.900308
dtype: float64

## Statistical Analysis
### Finding Errors

Now that we have determined the variables that we are interested in we should conduct statistical analysis to determine outliers and errors which can be reported back to the sensor owners. <br>
This analysis will also feed into any future plots

In [12]:
# Want to look to see if there are errors in the Density data
data=dataFrame['Density (kg/m3)']

# Simple Z-score to determine the number of standard deviations values are away from mean.
# Need to conduct the abs since we can have values lower and higher than the mean. 
zScore = np.abs(stats.zscore(data))

# Determine an array with index values where the abs(z-score) is greater than 3.  
zArray=np.where(zScore>3) 

densityStats={'Mean':np.mean(data), 
             'StandardDeviation':np.std(data),
             'UpperLimit': np.mean(data)+(3*np.std(data)),
             'LowerLimit': np.mean(data)-(3*np.std(data))}

# Create a dataFrame for these erroneous values.
# These will form part of the output to feedback to device owner.
errorFrameDensity=dataFrame.iloc[zArray[0],[3,11]]



In [13]:
# As above but for Salinity
data=dataFrame['Salinity (PSU)']

zScore = np.abs(stats.zscore(data))
zArray = np.where(zScore>3)

salinityStats={'Mean':np.mean(data), 
             'StandardDeviation':np.std(data),
             'UpperLimit': np.mean(data)+(3*np.std(data)),
             'LowerLimit': np.mean(data)-(3*np.std(data))}

errorFrameSalinity=dataFrame.iloc[zArray[0],[3,13]]

In [14]:
# As above but for Temperature
data=dataFrame['Temperature (degC)']

zScore = np.abs(stats.zscore(data))
zArray = np.where(zScore>3)

temperatureStats={'Mean':np.mean(data), 
             'StandardDeviation':np.std(data),
             'UpperLimit': np.mean(data)+(3*np.std(data)),
             'LowerLimit': np.mean(data)-(3*np.std(data))}

errorFrameTemperature=dataFrame.iloc[zArray[0],[3,7]]

## Reporting Errors

Export error dataFrame so that the sensor owner can investigate potential problems

In [ ]:
# Output all of the errorFrames into indicdual CSV to report back to sensor owner. 
errorFrameDensity.to_csv('Density_Errors.csv')
errorFrameSalinity.to_csv('Salinity_Errors.csv')
errorFrameTemperature.to_csv('Temperature_Errors.csv')
errorFrameTimes.to_csv('Time_Errors.csv')

# 4) Data Visualisation
### Stage 1 - Data/Plot Setup

In [15]:
# Create ColumnDataSource for Bokeh Wizardry. This allows plots to be interactive and linked together.
source=ColumnDataSource(dataFrame)

# We want to include the temperature to see if it plays an impact on the variables
# This could be relationship could be hidden by use of linear correlations only
# Choose the palette first, then select max and min values from temperatureStats dictionary
color_mapper = LinearColorMapper(palette='Magma256', low=temperatureStats['LowerLimit'],
                                 high=temperatureStats['UpperLimit'])
Colors = {'field': 'Temperature (degC)', 'transform': color_mapper}

# We want further information to pop up once the user mouses over the data points. 
# To do this we create Tooltips which will facilitate hover actions when mousing over data points.
# This will display the Date, Temperature and Depth to the user on hover.
toolTips = [('DateTime ','@{DateTimeLabels}'),
           ('Temperature ','@{Temperature (degC)}'),
           ('Depth ','@{Depth (m)}')] 

# Setup the tools that will be common to plots
tools="pan,lasso_select,box_select,hover,zoom_in,zoom_out, box_zoom, wheel_zoom,reset,save"

# Possible to change the marker size based on the depth here. 
# For larger data files this is not suitable so comment out the 1st line. 
Sizes=('Depth (m)')
# Sizes=4

### Stage 2 - Create Interactive Plots and Link Widgets 

In [16]:
# Define the columns of interest
strX1='Density (kg/m3)'
strY1='Salinity (PSU)'
strZ1='Depth (m)'
strDT='DateTime (yyyy-mm-ddThh:mm:ss.sss)'

# Line Below Produces HTML output suitable for inclusion onto a website
output_file('WCO-CTD-L4_Summaryfile.html')


# Initialise FILTER Widgets here
# These are widgets that filter the data for the user depending on input.
# They are treated different to other widgets and must sit here
dateSlider = DateRangeSlider(title="Filter Data by Date",
                             value=(min(dataFrame[strDT]), ((dataFrame[strDT])[1380])),
                             start=min(dataFrame[strDT]), end=max(dataFrame[strDT]))
depthSlider = RangeSlider(title="Filter Data by Depth (m)",
                          value=(min(dataFrame[strZ1]), (max(dataFrame[strZ1]))),
                          start=min(dataFrame[strZ1]), end=max(dataFrame[strZ1]))

# Create customJS filter which is used to select data on interest
# when the user moves the DateRange slider
dateFilter = CustomJSFilter(args=dict(slider=dateSlider), code="""
    const indices = []
    const date_from = slider.value[0];
    const date_to = slider.value[1]
    for (var i = 0; i < source.get_length(); i++) {
        if ((source.data['Timestamp'][i] <= date_to) && (source.data['Timestamp'][i] >= date_from)) {
            indices.push(true)
        } else {
            indices.push(false)
        }
    }
    return indices
""")

# Create customJS filter which is used to select data on interest
# when the user moves the DepthRange slider
depthFilter = CustomJSFilter(args=dict(slider1=depthSlider), code="""
    const indices = []
    const depth_from = slider1.value[0];
    const depth_to = slider1.value[1]
    for (var i = 0; i < source.get_length(); i++) {
        if ((source.data['Depth (m)'][i] <= depth_to) && (source.data['Depth (m)'][i] >= depth_from)) {
            indices.push(true)
        } else {
            indices.push(false)
        }
    }
    return indices
""")

# Code below detects a change when the sliders are moved which then forces the CustomJSFilters to run
dateSlider.js_on_change('value', CustomJS(args=dict(source=source), code="""
   source.change.emit()
"""))
depthSlider.js_on_change('value', CustomJS(args=dict(source=source), code="""
   source.change.emit()
"""))

# Think of this View as the original dataFrame filtered within the ranges of the sliders.
# This view is then used to update the plots
view = CDSView(source=source, filters=[dateFilter,depthFilter])


# Setup the plots below
# Box below indicates the region within 3 standard deviations for Density and Salinity. 
box = BoxAnnotation(bottom=salinityStats['LowerLimit'],top=salinityStats['UpperLimit'], 
                    right=densityStats['UpperLimit'], left=densityStats['LowerLimit'], 
                    fill_alpha=0.05, fill_color='#0072B2')

qr0=figure(title='Filtered Data- Autoscale', tools=tools,tooltips=toolTips)
qr0.add_layout(box)
qr0.scatter(x=strX1,y=strY1,source=source, size=4, color=Colors, hover_color='green',view=view)
qr0.xaxis.axis_label = strX1
qr0.yaxis.axis_label = strY1

qr2=figure(title='Filtered Data - Within 3SD',tools=tools,tooltips=toolTips)
qr2.scatter(x=strX1,y=strY1,source=source, size=4, color=Colors, hover_color='green', view=view)
qr2.x_range=Range1d(densityStats['LowerLimit'],densityStats['UpperLimit'])
qr2.y_range=Range1d(salinityStats['LowerLimit'],salinityStats['UpperLimit'])
qr2.xaxis.axis_label = strX1
qr2.yaxis.axis_label = strY1
qr2.add_layout(box)

qr3=figure(title='Unfiltered Data - User View',tools=tools,tooltips=toolTips)
points=qr3.scatter(x=strX1,y=strY1,source=source, size=Sizes, color=Colors, hover_color='green')
qr3.x_range=Range1d(1024,1028)
qr3.y_range=Range1d(32,37)
qr3.xaxis.axis_label = strX1
qr3.yaxis.axis_label = strY1

qr4=figure(title='Filtered Data - User View',tools=tools,tooltips=toolTips)
points=qr4.scatter(x=strX1,y=strY1,source=source, size=Sizes, color=Colors, hover_color='green', view=view)
qr4.x_range=Range1d(1024,1028)
qr4.y_range=Range1d(32,37)
qr4.xaxis.axis_label = strX1
qr4.yaxis.axis_label = strY1

# Initialise the colorBar and add it to two of the plots  
colorBar = ColorBar(color_mapper=color_mapper,location=(0, 0),  title='Temp')
qr0.add_layout(colorBar,'right')
qr3.add_layout(colorBar,'right')

# Initialise PLOT RANGE Widgets here
# These are widgets that adjust the graphical ranges.
# They rely on information from the plots above and therefore must live here
densityView= RangeSlider(title="Adjust Density (kg/m3) Range View",step=0.5, 
                         value=(qr4.x_range.start,qr4.x_range.end), start=1008, end=1028)
densityView.js_link("value", qr4.x_range, "start", attr_selector=0)
densityView.js_link("value", qr4.x_range, "end", attr_selector=1)
densityView.js_link("value", qr3.x_range, "start", attr_selector=0)
densityView.js_link("value", qr3.x_range, "end", attr_selector=1)

salinityView= RangeSlider(title="Adjust Salinity (PSU) Range View",step=0.5, 
                          value=(qr4.y_range.start,qr4.y_range.end), start=12, end=37)
salinityView.js_link("value", qr4.y_range, "start", attr_selector=0)
salinityView.js_link("value", qr4.y_range, "end", attr_selector=1)
salinityView.js_link("value", qr3.y_range, "start", attr_selector=0)
salinityView.js_link("value", qr3.y_range, "end", attr_selector=1)

# Bring it all together below
# Bring all of the widgets together into a layout
# Bring the plots and widgets into a layout
# Code which silences the FIXED_SIZING_MODE is require since the best place to put the 
# exact width and height of the widgets is not known. This is a hacky workaround
silence(FIXED_SIZING_MODE, True) 
widgets = column (row(dateSlider,depthSlider),
                  row(densityView,salinityView),
                  width=200, height=100, sizing_mode='fixed')
 
grid = gridplot(children = [[qr0, qr2], [qr3,qr4],[widgets] ], sizing_mode = 'stretch_both')

# Print to screen
show(grid)

# Moving Forward
### Hacky Things

- `FIXED_SIZING_MODE` Error is silenced. This error is raised during the plotting stage. It is raised because the exact location for the explicit height and width values for the widgets is not known by the author
- `dateSlider` Upper value for date has been hardcoded with index of `[1380]` to capture first 6 months 

### Developments to improve in future evolutions
- Analysis
    - At the moment the the only trends in the variables analysed are linear correlations by using the `corr()` method. Future iterations should look at non linear relationships.
    - Merge errorFrames together so that rows with multiple errors appear on one line only
- Interactivity
    - Include a Widget for the user to switch dynamic depth sizes on and off
    - Include a Widget to scroll through density/salinity data which will filter the data for one month across different years. This will allow the user to see if there are changes year to year for the same month 
    - Improve the Hover tool so that the long timestamp is removed and only the date is printed to the screen 
    - Add in an option for the user to export data from the filtered visualisations
- Efficiency
    - Improve the efficiency of the algorithms used, there is a lag of tens of seconds when updating the charts. This is possibly due to my laptop but I'm sure there are more efficient ways to handle the data
    - Replace all list functions with dataframe apply functions
- Visualisations
    - Replace two colorBars with one that is located on the left hand side but spans the two rows 